In [ ]:
from aml_obs.drift import Drift_Analysis
from aml_obs.collector import Online_Collector
from aml_obs.query import RT_Visualization
from azureml.core import Workspace
import pandas as pd
from azureml.core.model import Model
from sklearn.preprocessing import LabelEncoder
import joblib

ws = Workspace.from_config()
drift_analysis = Drift_Analysis(ws)

In [ ]:
# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_daemon(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(1000):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('today'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging_daemon()

In [ ]:
# from monitoring.query import RT_Visualization
streaming_table_name="streaming_test"
rt_viz =RT_Visualization(streaming_table_name,ws)
rt_viz.scatter(max_records=200, ago='12h',groupby='lr', y_metric='metric1',x_metric='timestamp')

In [ ]:
model = joblib.load("./model-deployment/model.joblib")

In [ ]:
df_test = pd.read_parquet("data/iris.parquet")

In [ ]:
X_test = df_test[['sepal_length','sepal_width','petal_length','petal_width']]
predictions = model.predict(X_test)
labels = ['setosa', 'versicolor', 'virginica']
encoder = LabelEncoder()
encoder.fit(labels)

In [ ]:
# could add logging demo here

import datetime
df1 = pd.DataFrame()
probs = model.predict_proba(X_test)
probs = [prob for prob in probs]
df1['probs'] = probs
preds = model.predict(X_test)
df1['pred'] = preds
df1['scoring_service'] = "managed_online"
ts =datetime.datetime.now()
df1['timestamp'] = ts
df1